<a href="https://colab.research.google.com/github/alex123012/burrows-wheeler-transformation/blob/master/CuPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U setuptools pip wheel
!pip install cupy-cuda101

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (47.1.1)
     |████████████████████████████████| 1.5MB 4.6MB/s 
Requirement already up-to-date: wheel in /usr/local/lib/python3.6/dist-packages (0.34.2)
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [0]:
import numpy as np
import cupy as cp

In [0]:
def BWT_trans(string):
    # Перевод строки в список чисел по юникоду
    str_list = [ord(i) for i in string]

    # Переход к массиву на GPU
    s_arr = cp.array(str_list)
    # Создание матрицы со сдвигом
    # Пояснение:
    # сдвиг матрицы происходит на устройстве, потом переводится в список на хост
    # и затем снова в массив на устройстве (делается из-за проблем с отсутствием
    # поддержки типа списков с нечисловыми элементами)
    s_mat = cp.array([cp.roll(s_arr, i).tolist() for i in range(0,len(s_arr))])
    # Сортировка по индексам столбцов
    # Пояснение: аналогично вышенаписанному
    i = cp.lexsort(cp.array(
        [s_mat[:,i].tolist() for i in range(len(s_arr)-1, -1, -1)]))
    s_mat = s_mat[i]
    
    # Обратный перевод чисел в симвлы по юникоду
    str_list = [chr(i) for i in s_mat[:,-1].tolist()]
    #Чтобы можно было и в переменную записать, и на экран сразу вывести
    print(''.join(str_list), cp.where(cp.all(s_arr==s_mat, axis=1))[0].item())
    return (''.join(str_list), cp.where(cp.all(s_arr==s_mat, axis=1))[0].item())

In [4]:
BWT_trans('abrakadabra')

rdakraaaabb 2


('rdakraaaabb', 2)

In [0]:
def BWT_rever(string, ind):
    # Перевод строки в список чисел по юникоду
    str_list = [ord(i) for i in string]

    # Переход к массиву на GPU
    # Массив с последовательностью введенной строки
    s_arr = cp.array(str_list)
    # отсортированная последовательность
    sorted_s = cp.array(sorted(str_list))
    # Просто слияние двух предыдущих массивов
    tab_s = cp.vstack((s_arr, sorted_s))
    for i in range(1, len(s_arr)-1):
        # Сортировка, с получением индексов
        # массива tab_s для ново1 строки, хотя фактически - столбца
        # (метод .T меняет оси массива местами)
        # Пояснение:
        # сдвиг матрицы происходит на устройстве, потом переводится в список на хост
        # и затем снова в массив на устройстве (делается из-за проблем с отсутствием
        # поддержки типа списков с нечисловыми элементами)
        j = cp.lexsort(cp.array([tab_s.T[:,i].tolist() for i in range(i, -1, -1)]))
        # Добавление отсортированной строки к массиву
        tab_s = cp.vstack((s_arr,tab_s.T[j].T))
    # Сортировка последней строки
    j = cp.lexsort(cp.array([tab_s.T[:,i].tolist() for i in range(len(s_arr)-1, -1, -1)]))

    # Обратный перевод чисел в симвлы по юникоду
    str_list = [chr(i) for i in tab_s.T[j][ind]]
    #Чтобы можно было и в переменную записать, и на экран сразу вывести
    print(''.join(str_list))
    return ''.join(str_list)

In [12]:
BWT_rever('rdakraaaabb', 2)

abrakadabra


'abrakadabra'